## NetworkFile from ICL Code, PNExtract To ARM input

In [671]:
import numpy as np
from pathlib import Path
import pandas as pd
import os

def Statoil_load(path, prefix):
    network = {}
    
    path = Path(path)
    filename = Path(path.resolve(), prefix + '_link1.dat')   
    with open(filename, mode='r') as f:
        link1 = pd.read_csv(filepath_or_buffer=f,
                              header=None,
                              skiprows=1,
                              sep=' ',
                              skipinitialspace=True,
                              index_col=0)
        link1.columns = ['throat.pore1', 'throat.pore2', 'throat.radius',
                             'throat.shape_factor', 'throat.total_length']
    network['throat_connc'] = np.vstack((link1['throat.pore1'], link1['throat.pore2'])).T 
    network['throat_len'] = np.vstack(link1['throat.total_length'])
#----------------------------------------------------------------------------#    


    filename = Path(path.resolve(), prefix + '_node2.dat')
    with open(filename, 'r') as f:
        node2 = pd.read_csv(f,header=None, 
                            sep=' ',
                            skipinitialspace=True,
                            index_col=0)
        node2.columns=['pore_volume', 'pore_radius', 
                       'pore_shape_factor', 'pore_clay_volume']
        network['pore_radius'] = np.vstack(node2['pore_radius'])
        
#----------------------------------------------------------------------------#  


    filename = Path(path.resolve(), prefix + '_node1.dat')
    with open(filename, 'r') as f:
        info = f.readline().split()
        nor = int(info[0])
        array = np.ndarray([nor,6])
        for i in range(nor):
            row = f.readline().replace('\t',' ').replace('\n',' ').split();
            array[i,:] = row[0:6]
        node1 = pd.DataFrame(array[:,[1,2,3,4]]);
        node1.columns = ['pore.x_coord', 'pore.y_coord', 'pore.z_coord',
                         'pore.coordination_number']
    network['pore_cords'] = np.vstack((node1['pore.x_coord'],node1['pore.y_coord'], node1['pore.z_coord'])).T
    network['pcn'] = np.vstack(node1['pore.coordination_number'])
    #----------------------------------------------------------------------------
    #Trimming the inlet and outlet throats 
    
    trim_net_in = np.where(np.any(network['throat_connc'] == -1, axis = 1))[0]
    inlets = network['throat_connc'][trim_net_in,1]
    remained_net_connc = np.delete(network['throat_connc'], trim_net_in, axis = 0)
    remained_net_len = np.delete(network['throat_len'], trim_net_in, axis = 0)
    
    trim_net_out = np.where(np.any(remained_net_connc == 0, axis = 1))[0]
    outlets = remained_net_connc[trim_net_out]
    final_net_connc = np.delete(remained_net_connc, trim_net_out, axis = 0)
    final_net_len = np.delete(remained_net_len, trim_net_out, axis = 0)
    
    network['throat_connc'] = np.vstack(final_net_connc);
    network['throat_len'] = np.vstack(final_net_len);
    network['inlets'] = np.vstack(inlets);
    network['outlets'] = np.vstack(outlets);
    
    return network;


def StatoilToARM(path, prefix, network):
    """The input format of network consists
    Throats(cylinders) and Pores(spheres).
    In ARM, Pores are cylindrical and there are no throats
    So network topology is based on the thoats as they are connection between pores"""
    'Input: Statoil generated network from ICL pn-extract code '
    ARM_pore = {}
    ARM_pore_info =  distributed_volume(net = network)   # array of calculated radius of ARM_pore and pore_cordination_number
    pore_data = pd.DataFrame(data = ARM_pore_info, columns = ['ARM_pore_radius', 'ARM_pcn'])

    ARM_pore['pore_data'] = np.vstack(( pore_data['ARM_pore_radius'].round(10), net['throat_len'].T.round(10), pore_data['ARM_pcn']))
    print(pore_data)
    path = Path(path)
    dir = Path(path.resolve(), prefix);
    if not os.path.exists(dir):
        os.makedirs(dir)
        
    filename = Path(dir.resolve(),prefix+'_pores_from_Rishabh')
    file = open(filename,'+w')
    file.write('# name \t d \t l \t b \t q \t Da_local \t G_local \n');
    file.write('# ---------------------------------------------------------------\n');
    
    data = pd.DataFrame(ARM_pore['pore_data'].T, dtype=float)
    data.to_csv(file, header=None,sep = '\t')
    file.close();

def distributed_volume(net):
    """This function distribute the volume of spherical pores(PnExtract philosophy) among the connections
    Input:
            Network obj
            
    Output:
            returns array[np.throat, 2] of 
                col1: radius of cylinderical pores(ARM philosophy), 
                col2: ARM_pore_cordination_number
            
            Radius of ARM_pore is equal to 
                                                    Volume of connected Spherical pores
            Volume of cylinderical ARM_pore = Sum( ----------------------------------------)
                                                        (pore_coordination_number)
                                                (4.pi.r1^3)                   (4.pi.r2^3)
            pi* r_throat^2 * l  =      ------------------------- +    ----------------------------
                                        (3 * pore_cordination no.)      (3 * pore_cordination no.)
    """
    
    pcn = net['pcn']
    data = np.zeros((len(net['throat_connc']),2));
    #Calculating distributed Volume for each ARM_pore   
    for i in range(len(net['throat_connc'])):
        Pore1, Pore2 = net['throat_connc'][i] - 1
        V1 = (4.0* np.pi * pow(net['pore_radius'][Pore1],3)/(3.0 * pcn[Pore1]))
        V2 = (4.0* np.pi * pow(net['pore_radius'][Pore2],3)/(3.0 * pcn[Pore2]))
        data[i,0] = np.sqrt((V1 + V2)/(2 * np.pi * net['throat_len'][i]))
        data[i,1] = (pcn[Pore1] + pcn[Pore2] - 2);
        
    return data
    
    
    
def WriteVTK(net):
    f = open('net.vtk','w+')
    f.write("# vtk DataFile Version 2.0\n Network example\nASCII\nDATASET POLYDATA\n")
    cords = net['pore_cords']
    connc = np.array(net['pore_connc'], dtype=np.int8)
    row = np.shape(cords)[0]
    f.write("POINTS %d float\n" %row)
    np.savetxt(f,cords)
    f.close();

In [672]:
path = 'PNExtract_network/Smerdyna2D'
net = Statoil_load(path = path,prefix='Smerdyna')
#print("".join(str(net['pore_cords'])))

In [673]:
outpath = 'PnextractToARM/'
StatoilToARM(path = outpath, prefix = 'Smerdyna', network = net)

      ARM_pore_radius  ARM_pcn
0        6.085021e-07      5.0
1        7.672725e-07      5.0
2        6.265701e-07      6.0
3        5.741387e-07      7.0
4        7.491053e-07      5.0
...               ...      ...
1182     5.626635e-07      6.0
1183     7.090706e-07      3.0
1184     9.476574e-07      4.0
1185     3.482568e-07      6.0
1186     7.219491e-07      4.0

[1187 rows x 2 columns]
